In [ ]:
import pandas as pd

In [2]:
df = pd.read_csv('./data/all_universe_predictions.csv')

In [ ]:
# 1. Mark each prediction as ambiguous
#df['is_ambiguous'] = df['pred_set'].apply(lambda labels: 1 if len(labels) > 1 else 0)

for CP threshold policy no difference, is it double in this data frame? Could this cause problems with fanova?

In [3]:
from ast import literal_eval

df['pred_set'] = df['pred_set'].apply(
    lambda x: literal_eval(x) if isinstance(x, str) else x
)

df['is_ambiguous'] = df['pred_set'].apply(lambda preds: 1 if len(preds) > 1 else 0)

# Melt the subgroup flags into one column so you can group by them

In [4]:
# List of binary subgroup columns
subgroup_cols = ['frau1', 'nongerman', 'nongerman_male', 'nongerman_female']

# Melt to long format: one row per (row, subgroup) where value == 1
df_long = df.melt(
    id_vars=['UniverseID', 'feature_set', 'model', 'is_ambiguous'], #'threshold_policy'
    value_vars=subgroup_cols,
    var_name='subgroup',
    value_name='is_member'
)

# Filter to keep only the subgroup memberships (i.e., rows where the person is in that group)
df_long = df_long[df_long['is_member'] == 1]

# in subgroup column it says "other 9%", for what does it stand? 


# Compute the fraction of ambiguous predictions per (universe, subgroup)

In [ ]:
group_cols = ['UniverseID', 'feature_set', 'model', 'subgroup'] #'threshold_policy',
fractions = df_long.groupby(group_cols)['is_ambiguous'].mean().reset_index()
fractions.rename(columns={'is_ambiguous': 'ambiguity_fraction'}, inplace=True)

ToDo: fractions has 12 universes, but only 6 are actually different because of CP
- Do I need to include overall in subgroups?

In [ ]:
# Keep only rows where UniverseID is an odd number
unique_universes = fractions[fractions['UniverseID'] % 2 == 1].reset_index(drop=True)
fractions = unique_universes


# For each universe, compute the maximum pairwise difference in ambiguity fraction between subgroups

In [ ]:
# Compute disparity per universe = max - min ambiguity_fraction across subgroups
disparity_df = fractions.groupby(['feature_set', 'model'])['ambiguity_fraction']\
                        .agg(lambda x: x.max() - x.min())\
                        .reset_index(name='ambiguity_disparity')

## add universe id 


# fANOVA

In [ ]:
import numpy as np
from ConfigSpace import ConfigurationSpace, CategoricalHyperparameter
from fanova import fANOVA

In [ ]:
# Define the ConfigSpace with categorical decision options
cs = ConfigurationSpace()
for col in ['feature_set', 'model']: # 'threshold_policy'
    choices = sorted(disparity_df[col].unique().tolist())
    cs.add(CategoricalHyperparameter(col, choices))


In [ ]:
# Encode categorical values as integer codes
#encoders = {
#    col: {val: idx for idx, val in enumerate(sorted(disparity_df[col].unique()))}
#    for col in ['feature_set', 'model', 'threshold_policy']
#}
#
#X = np.vstack([
#    disparity_df['feature_set'].map(encoders['feature_set']),
#    disparity_df['model'].map(encoders['model']),
#    disparity_df['threshold_policy'].map(encoders['threshold_policy']),
#]).T
#
#Y = disparity_df['ambiguity_disparity'].to_numpy()

In [ ]:
hp_order = [hp.name for hp in cs.values()]
X = disparity_df[hp_order].copy()

In [ ]:
# Map categorical values to integers (same as before)
for col in hp_order:
    encoder = {val: i for i, val in enumerate(sorted(disparity_df[col].unique()))}
    X[col] = X[col].map(encoder)

X_array = X.to_numpy(dtype=float)  # ensure float dtype


In [ ]:
Y = disparity_df['ambiguity_disparity'].to_numpy()

In [ ]:
# Run fANOVA
fanova = fANOVA(X_array, Y, config_space=cs)


In [ ]:
# Get individual importance scores
importance = {
    'feature_set': fanova.quantify_importance((0,))['individual importance'],
    'model': fanova.quantify_importance((1,))['individual importance'],
    #'threshold_policy': fanova.quantify_importance((2,))['individual importance'],
}

In [ ]:
# Get individual importance scores
# Check for valid dimension indices and handle empty or invalid results
try:
    importance = {
        'feature_set': fanova.quantify_importance((0,))['individual importance'],
        'model': fanova.quantify_importance((1,))['individual importance'],
        # 'threshold_policy': fanova.quantify_importance((2,))['individual importance'],
    }
except (IndexError, KeyError, RuntimeError) as e:
    print(f"Error computing importance: {e}")
    importance = {}


In [ ]:
# Inspect which dimensions are available in fANOVA
print("Available dimensions in V_U_total:")
print(list(fanova.V_U_total.keys()))

print("Total variance per tree (non-zero indices):")
print([i for i, v in enumerate(fanova.trees_total_variance) if v != 0])


In [ ]:
# Get individual importance scores with debug output
try:
    print("Available dimensions in V_U_total:", list(fanova.V_U_total.keys()))
    print("Non-zero total variances:", [i for i, v in enumerate(fanova.trees_total_variance) if v != 0])

    importance = {
        'feature_set': fanova.quantify_importance((0,))['individual importance'],
        'model': fanova.quantify_importance((1,))['individual importance'],
    }
except (IndexError, KeyError, RuntimeError) as e:
    print(f"Error computing importance: {e}")
    importance = {}


In [ ]:
# Deep inspection of V_U_total and trees_total_variance
sub_dims = (0,)
try:
    print(f"Length V_U_total{sub_dims} =", len(fanova.V_U_total[sub_dims]))
    print(f"Length trees_total_variance =", len(fanova.trees_total_variance))

    for i in range(len(fanova.V_U_total[sub_dims])):
        print(f"Tree {i}: V_U_total = {fanova.V_U_total[sub_dims][i]}, Total variance = {fanova.trees_total_variance[i]}")

    # Try quantifying importance
    importance = fanova.quantify_importance(sub_dims)['individual importance']
    print("Importance computed:", importance)

except Exception as e:
    print(f"Exception during inspection or importance computation: {e}")


In [ ]:
# Check for valid variance data before computing importance
sub_dims = (0,)

if sub_dims in fanova.V_U_total and len(fanova.V_U_total[sub_dims]) > 0:
    try:
        importance = fanova.quantify_importance(sub_dims)['individual importance']
        print("Importance computed:", importance)
    except Exception as e:
        print(f"Exception during importance computation: {e}")
        importance = None
else:
    print(f"No usable variance data for dimension {sub_dims}. Skipping.")
    importance = None


In [ ]:
# Safely compute importance for all expected dimensions
importance = {}
for i, name in enumerate(['feature_set', 'model']):
    sub_dims = (i,)
    if sub_dims in fanova.V_U_total and len(fanova.V_U_total[sub_dims]) > 0:
        try:
            imp = fanova.quantify_importance(sub_dims)['individual importance']
            importance[name] = imp
            print(f"Importance for {name}: {imp}")
        except Exception as e:
            print(f"Exception during importance computation for {name}: {e}")
            importance[name] = None
    else:
        print(f"No usable variance data for dimension {sub_dims} ({name}). Skipping.")
        importance[name] = None


In [ ]:
# Investigate full contents of V_U_total and trees_total_variance
print("All entries in V_U_total:")
for dims, values in fanova.V_U_total.items():
    print(f"  {dims}: length = {len(values)}")

print("Total number of trees with non-zero variance:", sum(v != 0 for v in fanova.trees_total_variance))


In [ ]:
# Inspect raw input variance to understand why fANOVA produced no usable importance values

# X should be the input matrix passed to fANOVA
variances = np.var(X, axis=0)
for i, v in enumerate(variances):
    print(f"Feature {i}: variance = {v}")


In [ ]:
# Check available attributes
print("Available attributes in fanova:", dir(fanova))

# Fallback if `n_dimensions` is not available
if hasattr(fanova, 'config_space'):
    print("Config space dimensions:", len(fanova.config_space.get_hyperparameters()))

# Attempt recomputation (alternative safe method)
try:
    fanova._compute_variances()  # Internal method that populates V_U_total
    print("Recomputation of variances triggered.")
except Exception as e:
    print(f"Error during variance recomputation: {e}")


In [ ]:
# Try inspecting trees from the_forest assuming it's an object with accessible attribute
try:
    trees = fanova.the_forest.trees
    print(f"Number of trees in the forest: {len(trees)}")
    for i, tree in enumerate(trees[:3]):
        print(f"Tree {i} type: {type(tree)}")
        print(f"Tree {i} attributes: {dir(tree)}")
except Exception as e:
    print(f"Error inspecting trees in the forest: {e}")


In [ ]:
# Use safe access method to probe attributes
from inspect import getmembers

# Try accessing all non-method members safely
try:
    members = getmembers(fanova.the_forest, lambda a: not callable(a))
    for name, value in members:
        if not name.startswith('__'):
            if isinstance(value, list):
                print(f"{name}: list of length {len(value)}")
            elif hasattr(value, '__len__'):
                print(f"{name}: len = {len(value)}")
            else:
                print(f"{name}: type = {type(value)}")
except Exception as e:
    print(f"Error while accessing members of fanova.the_forest: {e}")


In [ ]:
# Manually try to access attributes that may hold the trees
possible_attrs = [
    'rf', 'forest', '_forest', 'estimators_', 'base_forest', 'wrapped_forest', 'model_forest', 'raw_forest'
]

for attr in possible_attrs:
    try:
        value = getattr(fanova.the_forest, attr)
        print(f"{attr}: type={type(value)}, len={len(value) if hasattr(value, '__len__') else 'N/A'}")
    except AttributeError:
        continue
    except Exception as e:
        print(f"{attr}: error accessing attribute: {e}")


In [ ]:
# Print the string representation of the forest object to look for clues
print("fanova.the_forest representation:")
print(fanova.the_forest)


In [ ]:
# Try accessing pyrfr wrapper if available
try:
    import pyrfr
    if isinstance(fanova.the_forest, pyrfr.regression.fanova_forest):
        print("Confirmed: fanova.the_forest is a pyrfr fanova_forest instance.")
        print("Number of trees:", fanova.the_forest.num_trees())
        print("Number of dimensions:", fanova.the_forest.num_features())
except Exception as e:
    print(f"Error accessing pyrfr forest methods: {e}")


In [ ]:
# Print results
print("fANOVA results (variance explained in ambiguity disparity):")
for key, val in importance.items():
    print(f"  {key}: {val * 100:.2f}%")

# fANOVA v2

In [ ]:
import numpy as np
if not hasattr(np, 'float'):
    np.float = float  # Patch for fanova compatibility


In [ ]:
# Step 1: Build ConfigSpace and add hyperparameters
from ConfigSpace import ConfigurationSpace, CategoricalHyperparameter

cs = ConfigurationSpace()
cs.add(
    CategoricalHyperparameter("feature_set", sorted(disparity_df["feature_set"].unique())),
    CategoricalHyperparameter("model", sorted(disparity_df["model"].unique())),
    #CategoricalHyperparameter("threshold_policy", sorted(disparity_df["threshold_policy"].unique()))
)

# Step 2: Get correct hyperparameter order
hp_order = [hp.name for hp in list(cs.values())]

# Step 3: Encode categorical variables using consistent mapping
X_df = disparity_df[hp_order].copy()
for col in hp_order:
    encoder = {val: i for i, val in enumerate(sorted(X_df[col].unique()))}
    X_df[col] = X_df[col].map(encoder)
X_df = X_df.astype(float)

# Step 4: Now pass this labeled DataFrame directly to fANOVA
from fanova import fANOVA

Y = disparity_df["ambiguity_disparity"].to_numpy()
fanova = fANOVA(X_df, Y, config_space=cs)


In [ ]:
# Step 1: Extract hyperparameters in correct order (no deprecated method)
hp_list = list(cs.values())

importance_scores = {}

# Step 2: Loop through in order and extract individual importance scores
for i, hp in enumerate(hp_list):
    try:
        result = fanova.quantify_importance((i,))
        importance_scores[hp.name] = result["individual importance"]
    except Exception as e:
        print(f"Could not compute importance for {hp.name}: {e}")

# Step 3: Format and display
importance_df = pd.DataFrame.from_dict(
    importance_scores, orient="index", columns=["individual importance"]
)
importance_df.sort_values("individual importance", ascending=False, inplace=True)

print("fANOVA: Individual variance explained per decision factor (%):")
print((importance_df * 100).round(2))


In [ ]:
import numpy as np
print("Var(Y):", np.var(Y))
# extremly small variance, so not much to explain

In [ ]:
print("X shape:", X_df.shape)
print("Unique values per column:")
print(X_df.nunique())
